In [2]:
import json 
import pandas as pd 
import soft_kmeans_alg 
import new_utils 
import json 
import numpy as np 
from scipy.stats import kendalltau
from scipy.stats import mode
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from copkmeans.cop_kmeans import cop_kmeans

In [10]:
# j_values = [50, 200, 500]
# r_values = [0.1, 0.25, 0.5, 0.75, 0.9]
# m_values = range(50)  # From 0 to 49 (inclusive)
# temp_json_results_dir = 'temp_json_results/conjugate_priors'
# not_available_fnames_file = 'results/not_available_fnames.txt'

# dic = {}
# dic['param'] = {
#     "num_of_datasets_per_combination": 50,
#     "n_iter": 5000,
#     "n_biomarkers": 10
# }
# not_available_fnames = []
# not_available_count = 0
# for j in j_values:
#     for r in r_values:
#         combstr = f"{int(j*r)}|{j}"
#         if combstr not in dic:
#             dic[combstr] = []
#         for m in m_values:
#             try:
#                 with open(f"{temp_json_results_dir}/temp_results_{j}_{r}_{m}.json") as f:
#                     d = json.load(f)
#                 tau = list(d.values())[0][0]
#                 dic[combstr].append(tau)
#             except:
#                 not_available_count += 1
#                 fname = f"{j} {r} {m}"
#                 not_available_fnames.append(fname)
#                 dic[combstr].append(np.nan)

# print(f"not available: {not_available_count}")
# with open('results/cp_results.json', "w") as file:
#         json.dump(dic, file, indent=4)

In [11]:
# with open(not_available_fnames_file, "w") as f:
#     for item in not_available_fnames:
#         f.write(f"{item}\n")

In [12]:
# with open(not_available_fnames_file, 'r') as f: 
#     loaded_list = [l.strip() for l in f]
# loaded_list

In [13]:
# js = [50, 200, 500]
# rs = [0.1, 0.25, 0.5, 0.75, 0.9]
# num_of_datasets_per_combination = 50
# alist = [
#         f"{int(j*r)}|{j}_{m}"
#         for j in js
#         for r in rs
#         for m in range(num_of_datasets_per_combination)
#     ]

In [14]:
# alist

In [15]:
# with open('temp_json_results/temp_results_50_0.1_0.json') as f:
#     d = json.load(f)
#     print(d)
# print(list(d.values())[0][0])

In [16]:
# # Define the ranges

# j_values = [50, 200, 500]
# r_values = [0.1, 0.25, 0.5, 0.75, 0.9]
# m_values = range(50)  # From 0 to 49 (inclusive)
# # Generate all combinations
# combinations = [f"{j} {r} {m}" for j in j_values for r in r_values for m in m_values]

# all_combinations_file = 'all_combinatinos.txt'
# with open(all_combinations_file, "w") as f:
#     for item in combinations:
#         f.write(f"{item}\n")
# # # Print the result
# # for combo in combinations:
# #     print(combo)

In [17]:
# n_shuffle = 2
# iterations = 100
# burn_in = 10
# thining = 2

In [3]:
data = pd.read_csv("data/5|50_2.csv")
data.head()
n_biomarkers = len(data.biomarker.unique())
biomarker_df = data[data['biomarker'] == "HIP-FCI (1)"].reset_index(drop=True)
biomarker_df.sample(10)

,participant,biomarker,measurement,diseased
6,6,HIP-FCI (1),-0.810355,True
15,15,HIP-FCI (1),-9.149381,True
7,7,HIP-FCI (1),-16.961797,True
35,35,HIP-FCI (1),2.340052,False
43,43,HIP-FCI (1),-14.110980,True
30,30,HIP-FCI (1),-7.951836,True
46,46,HIP-FCI (1),-8.622931,True
2,2,HIP-FCI (1),4.281634,True
22,22,HIP-FCI (1),-2.216792,True
13,13,HIP-FCI (1),-13.139134,True


In [4]:
n_init_value = int(10)
clustering_setup = KMeans(n_clusters=2, n_init=n_init_value)

# you need to make sure each measurment is a np.array before putting it into "fit"
measurements = np.array(biomarker_df['measurement']).reshape(-1, 1)

# Fit clustering method
clustering_result = clustering_setup.fit(measurements)
predictions = clustering_result.labels_

In [5]:
predictions

array([1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0], dtype=int32)

In [7]:
counts = np.bincount(predictions)
counts

array([25, 25])

In [28]:
all(count > 1 for count in counts)

True

In [24]:
# dataframe for non-diseased participants
healthy_df = biomarker_df[biomarker_df['diseased'] == False]
diseased_df = biomarker_df[biomarker_df['diseased'] == True]
# healthy_measurements = np.array(healthy_df['measurement']).reshape(-1, 1)
# which cluster are healthy participants in
healthy_predictions = predictions[healthy_df.index]
diseased_predictions = predictions[diseased_df.index]

In [25]:
healthy_predictions, diseased_predictions

(array([1, 1, 1, 1, 1], dtype=int32),
 array([0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        1], dtype=int32))

## Constrained kmeans

In [4]:
healthy_df = biomarker_df[biomarker_df['diseased'] == False]
diseased_df = biomarker_df[biomarker_df['diseased'] == True]
measurements = np.array(biomarker_df['measurement']).reshape(-1, 1)

In [5]:
healthy_df.index

Index([1, 12, 28, 35, 39], dtype='int64')

In [6]:
must_link = [(x, 0) for x in healthy_df.index]
must_link

[(1, 0), (12, 0), (28, 0), (35, 0), (39, 0)]

In [14]:
# input_matrix = np.random.rand(100, 1)
# input_matrix

In [7]:
clusters, centers = cop_kmeans(dataset=measurements, k=2, ml=must_link)
predictions = np.array(clusters)

In [8]:
predictions

array([1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0])

In [30]:
healthy_predictions = predictions[healthy_df.index]
diseased_predictions = predictions[diseased_df.index]

In [31]:
healthy_predictions

array([1, 1, 1, 1, 1])

In [7]:
# all_current_accepted_order_dicts = soft_kmeans_alg.metropolis_hastings_soft_kmeans(
#     data,
#     iterations,
#     n_shuffle,
# )

In [8]:
# new_utils.save_heatmap(all_current_accepted_order_dicts,
#                        burn_in, thining, folder_name='img',
#                        file_name='test_heatmap', title='heatmap')

In [9]:
# most_likely_order_dic = new_utils.obtain_most_likely_order_dic(
#     all_current_accepted_order_dicts, burn_in, thining)
# most_likely_order = list(most_likely_order_dic.values())
# tau, p_value = kendalltau(most_likely_order, range(1, n_biomarkers + 1))
# dic = {}
# dic['5|50'] = []
# dic['5|50'].append(tau)
# # write the JSON to a file
# with open("results.json", "w") as file:
#     json.dump(dic, file, indent=4)

In [10]:
# new_utils.get_theta_phi_estimates(data)

In [11]:
# biomarker_df.measurement.values

In [12]:
# # reshape to satisfy sklearn requirements
# measurements = np.array(biomarker_df['measurement']).reshape(-1, 1)

# # dataframe for non-diseased participants
# healthy_df = biomarker_df[biomarker_df['diseased'] == False]


In [13]:
# kmeans_setup = KMeans(2, random_state=0, n_init="auto")
# hierarchical_clustering_setup = AgglomerativeClustering(n_clusters=2)

In [14]:
# clustering_result_kmeans = kmeans_setup.fit(measurements)
# clustering_result_kmeans.labels_

In [15]:
# clustering_result_kmeans.predict(measurements)

In [16]:
# theta_mean, theta_std, phi_mean, phi_std = utils.compute_theta_phi_for_biomarker(
#     data, "HIP-FCI (1)", clustering_setup = None)

In [17]:
# theta_mean, theta_std, phi_mean, phi_std

In [18]:
# biomarkers = data.biomarker.unique()
# utils.get_theta_phi_estimates(data, biomarkers)